In [1]:
import os
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
import os
import librosa
import pandas as pd
import librosa.display
import soundfile as sf

최 = []
유 = []
경 = []
최_label = []
유_label = []
경_label = []

In [2]:
def load_wave_generator(path): 
       
    batch_waves = []
    labels = []
    # input_width=CHUNK*6 # wow, big!!
    folders = os.listdir(path)
    # folders = path
    #while True:
       # print("loaded batch of %d files" % len(files))
    for folder in folders:
        if not os.path.isdir(path):continue               #폴더가 아니면 continue                   
        files = os.listdir(path+"/"+folder)        
        print("Foldername :",folder,", - file count : ",len(files))         #폴더 이름과 그 폴더에 속하는 파일 갯수 출력
        if(folder == "0"):
            for wav in files:
                if not wav.endswith(".wav"):continue
                else:
                    global 최,최_label            #전역변수를 사용하겠다.
                    print("Filename :",wav)          #.wav 파일이 아니면 continue
                    y, sr = librosa.load(path+"/"+folder+"/"+wav)
                    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=45, hop_length=int(sr*0.01),n_fft=int(sr*0.02)).T


                    if(len(최)==0):
                        최 = mfcc
                        최_label = np.full(len(mfcc), int(folder))
                    else:
                        최 = np.concatenate((최, mfcc), axis = 0)
                        최_label = np.concatenate((최_label, np.full(len(mfcc),  int(folder))), axis = 0)
                        #print("mfcc :",mfcc.shape)
        if(folder == "1"):
            for wav in files:
                if not wav.endswith(".wav"):continue
                else:
                    global 유,유_label
                    print("Filename :",wav)
                    y, sr = librosa.load(path+"/"+folder+"/"+wav)
                    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=45, hop_length=int(sr*0.01),n_fft=int(sr*0.02)).T


                    if(len(유)==0):
                        유 = mfcc
                        유_label = np.full(len(mfcc), int(folder))
                    else:
                        유 = np.concatenate((유, mfcc), axis = 0)
                        유_label = np.concatenate((유_label, np.full(len(mfcc),  int(folder))), axis = 0)
                        #print("mfcc :",mfcc.shape)
        if(folder == "2"):
            for wav in files:
                if not wav.endswith(".wav"):continue
                else:
                    global 경, 경_label#전역변수를 사용하겠다.
                    print("Filename :",wav)#.wav 파일이 아니면 continue
                    y, sr = librosa.load(path+"/"+folder+"/"+wav)
                    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=45, hop_length=int(sr*0.01),n_fft=int(sr*0.02)).T


                    if(len(경)==0):
                        경 = mfcc
                        경_label = np.full(len(mfcc), int(folder))
                    else:
                        경 = np.concatenate((경, mfcc), axis = 0)
                        경_label = np.concatenate((경_label, np.full(len(mfcc),  int(folder))), axis = 0)
                        #print("mfcc :",mfcc.shape)

In [3]:

load_wave_generator("../sound_data")    #일단 RYU 음성 읽어오기


Foldername : .ipynb_checkpoints , - file count :  0
Foldername : 0 , - file count :  20
Filename : 1.wav
Filename : 10.wav
Filename : 11.wav
Filename : 12.wav
Filename : 13.wav
Filename : 14.wav
Filename : 15.wav
Filename : 16.wav
Filename : 17.wav
Filename : 18.wav
Filename : 19.wav
Filename : 2.wav
Filename : 20.wav
Filename : 3.wav
Filename : 4.wav
Filename : 5.wav
Filename : 6.wav
Filename : 7.wav
Filename : 8.wav
Filename : 9.wav
Foldername : 1 , - file count :  20
Filename : Ryu1.wav
Filename : Ryu10.wav
Filename : Ryu11.wav
Filename : Ryu12.wav
Filename : Ryu13.wav
Filename : Ryu14.wav
Filename : Ryu15.wav
Filename : Ryu16.wav
Filename : Ryu17.wav
Filename : Ryu18.wav
Filename : Ryu19.wav
Filename : Ryu2.wav
Filename : Ryu20.wav
Filename : Ryu3.wav
Filename : Ryu4.wav
Filename : Ryu5.wav
Filename : Ryu6.wav
Filename : Ryu7.wav
Filename : Ryu8.wav
Filename : Ryu9.wav
Foldername : 2 , - file count :  20
Filename : 경재원1.wav
Filename : 경재원10.wav
Filename : 경재원11.wav
Filename : 경재원12

In [11]:
# Dataset 상속
class CustomDataset(Dataset): 
  def __init__(self, x, y, transform):
    self.x_data = x
    self.y_data = y
    self.transform = transform

  # 총 데이터의 개수를 리턴
  def __len__(self): 
    return len(self.x_data)

  # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
  def __getitem__(self, idx): 
    x = torch.FloatTensor(self.x_data[idx])
    y = torch.FloatTensor(self.y_data[idx])
    return x, y

In [4]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
latent_size = 64
hidden_size = 256
image_size = 45
num_epochs = 200
batch_size = 100
sample_dir = 'samples'

# Create a directory if not exists
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

In [4]:
x = np.concatenate([최,유,경])
y = np.concatenate([최_label,유_label,경_label])
x.shape, y.shape

((49326, 45), (49326,))

In [7]:
np.save('./연습폴더/최유경_data', x)

In [8]:
np.save('./연습폴더/최유경_label', y)

In [124]:
x.T[0][1244]

-402.04608

In [137]:
y[45515]

2

In [12]:
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.5],   # 1 for greyscale channels
                                     std=[0.5])])

dataset = CustomDataset(x.T, y, transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [14]:
for i, (wav, _) in enumerate(dataloader):
#     wav = wav.reshape(batch_size, -1).to(device)
    print(i)
    print(wav.shape)
    print(_.shape)
    break

0
torch.Size([45, 49326])
torch.Size([45, 0])


In [144]:
for i, (wav, _) in enumerate(data_loader):
    wav = wav.reshape(batch_size, -1).to(device)
    print(i)
    print(wav.shape)
    print(_.shape)
    if i == 2:
        break
        
        
# 시발 어떻게 하는거야?

0
torch.Size([100, 784])
torch.Size([100])
1
torch.Size([100, 784])
torch.Size([100])
2
torch.Size([100, 784])
torch.Size([100])


In [141]:
# Discriminator
D = nn.Sequential(
    nn.Linear(image_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid())

# Generator 
G = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size),
    nn.Tanh())

# Device setting
D = D.to(device)
G = G.to(device)

# Binary cross entropy loss and optimizer
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0002)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0002)

def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()

# Start training
total_step = len(data_loader)
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(dataloader):
        images = images.reshape(batch_size, -1).to(device)
        
        # Create the labels which are later used as input for the BCE loss
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        # ================================================================== #
        #                      Train the discriminator                       #
        # ================================================================== #

        # Compute BCE_Loss using real images where BCE_Loss(x, y): - y * log(D(x)) - (1-y) * log(1 - D(x))
        # Second term of the loss is always zero since real_labels == 1
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
        
        # Compute BCELoss using fake images
        # First term of the loss is always zero since fake_labels == 0
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        # Backprop and optimize
        d_loss = d_loss_real + d_loss_fake
        reset_grad()
        d_loss.backward()
        d_optimizer.step()
        
        # ================================================================== #
        #                        Train the generator                         #
        # ================================================================== #

        # Compute loss with fake images
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        
        # We train G to maximize log(D(G(z)) instead of minimizing log(1-D(G(z)))
        # For the reason, see the last paragraph of section 3. https://arxiv.org/pdf/1406.2661.pdf
        g_loss = criterion(outputs, real_labels)
        
        # Backprop and optimize
        reset_grad()
        g_loss.backward()
        g_optimizer.step()
        
        if (i+1) % 200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                  .format(epoch, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(), 
                          real_score.mean().item(), fake_score.mean().item()))
    
    # Save real images
    if (epoch+1) == 1:
        images = images.reshape(images.size(0), 1, 28, 28)
        save_image(denorm(images), os.path.join(sample_dir, 'real_images.png'))
    
    # Save sampled images
    fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
    save_image(denorm(fake_images), os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch+1)))

# Save the model checkpoints 
torch.save(G.state_dict(), 'G.ckpt')
torch.save(D.state_dict(), 'D.ckpt')

RuntimeError: shape '[100, -1]' is invalid for input of size 2219670